In [11]:
import scipy.io as io

import numpy as np
import robustsp as rsp
from scipy.optimize import least_squares as lsq

def arma_est_bip_mm(x,p,q):
    bip_s_est = rsp.arma_est_bip_s(x,p,q) #inno scale shouldnt be 0
    
    beta_hat_s = np.array([*bip_s_est['ar_coeffs'], *bip_s_est['ma_coeffs']])
    
    N = len(x)
    
    F_mm = lambda beta: 1/(N-p)*rsp.muler_rho2(rsp.arma_s_resid(x,beta,p,q)[0])/bip_s_est['inno_scale']
    
    F_bip_mm = lambda beta: 1/(N-p)*rsp.muler_rho2(rsp.bip_s_resid(x,beta,p,q)[0])/bip_s_est['inno_scale']
    
    print(beta_hat_s)
    print(F_mm([-.81]))
    
    beta_arma_mm = lsq(F_mm, -beta_hat_s,xtol=5*1e-7,ftol=5*1e-7,method='lm')['x']
    
    beta_bip_mm = lsq(F_bip_mm, -beta_hat_s,xtol=5*1e-7,ftol=5*1e-7,method='lm')['x']
    
    a = arma_s_resid(x, beta_arma_mm, p, q)[0]
    
    a_sc = rsp.m_scale(a) # innovations m-scale for ARMA model
    
    a_bip = rsp.bip_s_resid(x, beta_bip_mm, p, q)[0]
    
    a_bip_sc = rsp.m_scale(a_bip) # innovations m-scale for BIP-ARMA model
    
    # final parameter estimate uses the model that provides smallest m_scale
    beta_hat = beta_arma_mm if a_sc<a_bip_sc else beta_bip_mm
    
    # final m-scale
    a_m_sc = min(a_sc, a_bip_sc)
    
    # Output the results
    
    results = {'ar_coeffs': -beta_hat[:p],
               'ma_coeffs': -beta_hat[p:],
               'inno_scale': a_m_sc,
               'cleaned_signal': bip_s_est['cleaned_signal'],
               'ar_coeffs_init': bip_s_est['ar_coeffs'],
               'ma_coeffs_init': bip_s_est['ma_coeffs']}
    
    return results
    
x = io.loadmat('x_Ao')['x_ao'].flatten()
p=1
q=0
arma_est_bip_mm(x,1,0)

[-0.63868856]
[nan inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf 

c:\users\computer\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in true_divide
  
c:\users\computer\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in true_divide
  


ValueError: Residuals are not finite in the initial point.